In [1]:
import sys
from pathlib import Path
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
try:
    import joblib
    import pandas as pd
    import numpy as np
except ImportError as e:
    logging.error(f"Erro ao importar bibliotecas: {e}")
    sys.exit(1)

In [4]:
src_path = Path("..") / "src"
sys.path.append(str(src_path.resolve()))

In [5]:
try:
    from preprocessador import processamento_teste
except ImportError as e:
    logging.error(f"Erro ao importar o módulo de processamento: {e}")
    sys.exit(1)

In [6]:
def carregar_modelo(caminho_modelo: str):
    try:
        modelo = joblib.load(caminho_modelo)
        logging.info("Modelo carregado com sucesso.")
        return modelo
    except Exception as e:
        logging.error(f"Erro ao carregar o modelo: {e}")
        return None

In [7]:
class GeradorDeDados:
    def __init__(self, seed: int = 101):
        self.seed = seed
        np.random.seed(self.seed)

    def gerar_dados_aleatorios(self, n_linhas: int) -> pd.DataFrame:
        """
        Gera um DataFrame com valores aleatórios para testar o modelo.
        """
        data = {
            'Age': np.random.randint(18, 80, size=n_linhas),
            'Sex': np.random.choice(['M', 'F'], size=n_linhas),
            'ChestPainType': np.random.choice(['TA', 'ATA', 'NAP', 'ASY'], size=n_linhas),
            'RestingBP': np.random.randint(90, 180, size=n_linhas),
            'Cholesterol': np.random.randint(100, 300, size=n_linhas),
            'FastingBS': np.random.randint(0, 2, size=n_linhas),
            'RestingECG': np.random.choice(['Normal', 'ST', 'LVH'], size=n_linhas),
            'MaxHR': np.random.randint(60, 200, size=n_linhas),
            'ExerciseAngina': np.random.choice(['Y', 'N'], size=n_linhas),
            'Oldpeak': np.random.uniform(0.0, 6.0, size=n_linhas),
            'ST_Slope': np.random.choice(['Up', 'Flat', 'Down'], size=n_linhas)
        }
        df = pd.DataFrame(data)
        logging.info("Dados aleatórios gerados com sucesso.")
        return df

In [8]:
gerador = GeradorDeDados(seed=101)

In [9]:
n_linhas_teste = 10
teste_dados = gerador.gerar_dados_aleatorios(n_linhas_teste)
teste_dados

2024-08-28 16:57:35,679 - INFO - Dados aleatórios gerados com sucesso.


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,49,M,TA,149,108,1,Normal,184,N,0.722602,Up
1,29,M,ATA,124,162,0,Normal,79,N,3.000700,Flat
2,35,F,TA,134,264,0,Normal,90,Y,0.828057,Up
3,24,M,ATA,162,199,1,LVH,70,N,0.316850,Down
4,41,F,ASY,109,128,0,Normal,163,Y,1.069662,Up
5,79,M,ASY,100,163,0,ST,80,N,2.654209,Flat
6,29,M,NAP,166,107,1,Normal,68,Y,5.265524,Down
7,65,M,TA,177,238,1,ST,146,N,5.695585,Flat
8,27,M,ASY,90,280,1,ST,182,N,2.869005,Up
9,31,F,ATA,163,284,1,Normal,86,N,2.766716,Up


In [10]:
colunas_label = ['Sex', 'ExerciseAngina']  
colunas_hot = ['ChestPainType', 'RestingECG', 'ST_Slope'] 

In [11]:
try:
    teste_dados_processados = processamento_teste(dataset=teste_dados, colunas_label=colunas_label, colunas_hot=colunas_hot)
    logging.info("Dados processados com sucesso.")
except Exception as e:
    logging.error(f"Erro ao processar os dados: {e}")
    sys.exit(1)

teste_dados_processados

2024-08-28 16:57:35,733 - INFO - Dados processados com sucesso.


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ST_Slope_Flat,ST_Slope_Up
0,49,1,149,108,1,184,0,0.722602,0,0,1,1,0,0,1
1,29,1,124,162,0,79,0,3.000700,1,0,0,1,0,1,0
2,35,0,134,264,0,90,1,0.828057,0,0,1,1,0,0,1
3,24,1,162,199,1,70,0,0.316850,1,0,0,0,0,0,0
4,41,0,109,128,0,163,1,1.069662,0,0,0,1,0,0,1
5,79,1,100,163,0,80,0,2.654209,0,0,0,0,1,1,0
6,29,1,166,107,1,68,1,5.265524,0,1,0,1,0,0,0
7,65,1,177,238,1,146,0,5.695585,0,0,1,0,1,1,0
8,27,1,90,280,1,182,0,2.869005,0,0,0,0,1,0,1
9,31,0,163,284,1,86,0,2.766716,1,0,0,1,0,0,1


In [12]:
colunas_drop = ['Cholesterol', 'ChestPainType_TA', 'RestingECG_Normal', 'RestingECG_ST']
for col in colunas_drop:
    if col in teste_dados_processados.columns:
        teste_dados_processados.drop(columns=[col], inplace=True)
        logging.info(f"Coluna {col} removida.")

2024-08-28 16:57:35,758 - INFO - Coluna Cholesterol removida.
2024-08-28 16:57:35,761 - INFO - Coluna ChestPainType_TA removida.
2024-08-28 16:57:35,763 - INFO - Coluna RestingECG_Normal removida.
2024-08-28 16:57:35,767 - INFO - Coluna RestingECG_ST removida.


In [13]:
modelo = carregar_modelo("../modelo/modelo_completo.joblib")
if modelo is not None:
    try:
        previsoes = modelo.predict(teste_dados_processados)
        print("Previsões:", previsoes)
    except Exception as e:
        logging.error(f"Erro ao fazer previsões: {e}")
else:
    logging.error("Modelo não carregado. Não foi possível fazer previsões.")

2024-08-28 16:57:35,954 - INFO - Modelo carregado com sucesso.


Previsões: [0 0 0 0 0 1 1 1 0 0]
